In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow import keras
import os

2024-03-29 14:31:43.395510: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-29 14:31:44.126892: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
def getImage(path: str, target_size=(250, 250)):
    image = tf.image.decode_jpeg(tf.io.read_file(path), channels=3)
    image = tf.image.resize(image, target_size)
    return image

def getData():
    path = os.path.join(os.path.dirname(os.getcwd()), "Rice_Image_Dataset")
    data = []
    labels = []
    for directory in os.listdir(path):
        dir_path = os.path.join(path, directory)
        if not os.path.isdir(dir_path):
            continue

        for img in os.listdir(dir_path):
            img_path = os.path.join(dir_path, img)
            data.append(img_path)
            labels.append(['Ipsala', 'Basmati', 'Karacadag', 'Arborio', 'Jasmine'].index(directory) + 1)

    dataset = tf.data.Dataset.from_tensor_slices((data, labels))
    dataset = dataset.map(lambda x, y: (getImage(x), tf.one_hot(y, depth=4)))
    return dataset.batch(32)

In [3]:
dataset = getData()
#muista sekottaa
#jaa trainig ja testi dataan


2024-03-29 14:31:45.260967: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-03-29 14:31:45.520591: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-03-29 14:31:45.520737: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

In [4]:
inputs = keras.Input(shape=(250, 250, 3))
x = keras.layers.Rescaling(1./255)(inputs)
x = keras.layers.Conv2D(filters=32, kernel_size=3, activation="relu")(x)
x = keras.layers.MaxPooling2D(pool_size=2)(x)
x = keras.layers.Conv2D(filters=64, kernel_size=3, activation="relu")(x)
x = keras.layers.MaxPooling2D(pool_size=2)(x)
x = keras.layers.Conv2D(filters=128, kernel_size=3, activation="relu")(x)
x = keras.layers.MaxPooling2D(pool_size=2)(x)
x = keras.layers.Conv2D(filters=256, kernel_size=3, activation="relu")(x)
x = keras.layers.MaxPooling2D(pool_size=2)(x)
x = keras.layers.Conv2D(filters=256, kernel_size=3, activation="relu")(x)
x = keras.layers.Flatten()(x)

outputs = keras.layers.Dense(4, activation="softmax")(x)

model = keras.Model(inputs=inputs, outputs=outputs)
model.compile(loss="categorical_crossentropy", optimizer='adam', metrics=["accuracy"])

In [5]:
history = model.fit(dataset, epochs=10)

Epoch 1/10


I0000 00:00:1711715506.481423    7306 service.cc:145] XLA service 0x7fda5c00d010 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1711715506.481467    7306 service.cc:153]   StreamExecutor device (0): NVIDIA GeForce RTX 3060 Ti, Compute Capability 8.6
2024-03-29 14:31:46.536525: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-03-29 14:31:46.952460: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:458] Loaded runtime CuDNN library: 8.5.0 but source was compiled with: 8.9.6.  CuDNN library needs to have matching major version and equal or higher minor version. If using a binary install, upgrade your CuDNN library.  If building from sources, make sure the library loaded at runtime is compatible with the version specified during compile configuration.
2024-03-29 14:31:46.964745: E external/local_xla/xla/stream_executor

FailedPreconditionError: Graph execution error:

Detected at node StatefulPartitionedCall defined at (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main

  File "<frozen runpy>", line 88, in _run_code

  File "/home/akseli/Asiakirjat/ohjelmointi/ml/myenv/lib/python3.11/site-packages/ipykernel_launcher.py", line 18, in <module>

  File "/home/akseli/Asiakirjat/ohjelmointi/ml/myenv/lib/python3.11/site-packages/traitlets/config/application.py", line 1075, in launch_instance

  File "/home/akseli/Asiakirjat/ohjelmointi/ml/myenv/lib/python3.11/site-packages/ipykernel/kernelapp.py", line 739, in start

  File "/home/akseli/Asiakirjat/ohjelmointi/ml/myenv/lib/python3.11/site-packages/tornado/platform/asyncio.py", line 205, in start

  File "/usr/lib/python3.11/asyncio/base_events.py", line 607, in run_forever

  File "/usr/lib/python3.11/asyncio/base_events.py", line 1922, in _run_once

  File "/usr/lib/python3.11/asyncio/events.py", line 80, in _run

  File "/h